In [1]:
import requests
import os


key = os.environ["GOOGLE_API_KEY"]
API_KEY = "&key="+str(key)
origin = "origin=SanAntonio, TX"
destination = "&destination=Oxnard, CA"
#include the prefix 'via:' to influence route but avoid stopovers
waypoints_south = "&waypoints=optimize:true|Andrews, TX|Yuma, AZ"
waypoints_north = "&waypoints=optimize:true|Andrews, TX|Flagstaff, AZ"
avoid="&avoid=tolls|ferries"
units="&units=imperial"


url_south = "https://maps.googleapis.com/maps/api/directions/json?" + origin + destination + waypoints_south + avoid + units + API_KEY
url_north = "https://maps.googleapis.com/maps/api/directions/json?" + origin + destination + waypoints_north + avoid + units + API_KEY

payload = {}
headers = {}

response_south = requests.request("GET", url_south, headers=headers, data=payload)
response_north = requests.request("GET", url_north, headers=headers, data=payload)
#response_json = response.json()
print(response_south.text)

{
   "geocoded_waypoints" : [
      {
         "geocoder_status" : "OK",
         "partial_match" : true,
         "place_id" : "ChIJrw7QBK9YXIYRvBagEDvhVgg",
         "types" : [ "locality", "political" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJWxSmF4yO-4YRy3h6AOIlOCo",
         "types" : [ "locality", "political" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJ2a1NIYdi1oARWEKe9Qc6qc8",
         "types" : [ "locality", "political" ]
      },
      {
         "geocoder_status" : "OK",
         "place_id" : "ChIJn2clE-ZN6IARRbZe-sBJgFk",
         "types" : [ "locality", "political" ]
      }
   ],
   "routes" : [
      {
         "bounds" : {
            "northeast" : {
               "lat" : 34.2221594,
               "lng" : -98.4945944
            },
            "southwest" : {
               "lat" : 29.4251833,
               "lng" : -119.17705
            }
         },
         "copyrights" : "Map data ©2023 Go

In [2]:
import json

data_south = json.loads(response_south.text)
data_north = json.loads(response_north.text)


def extract_start_locations(data):
    start_locations = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == 'start_location':
                start_locations.append(value)
            else:
                start_locations.extend(extract_start_locations(value))
    elif isinstance(data, list):
        for item in data:
            start_locations.extend(extract_start_locations(item))
    return start_locations

south_locs = extract_start_locations(data_south)
north_locs = extract_start_locations(data_north)
        
print(south_locs)
#print(n_start_locations)

[{'lat': 29.4251833, 'lng': -98.4945944}, {'lat': 29.4251833, 'lng': -98.4945944}, {'lat': 29.429325, 'lng': -98.49453799999999}, {'lat': 29.4297325, 'lng': -98.49934309999999}, {'lat': 29.4305865, 'lng': -98.5000136}, {'lat': 29.4369394, 'lng': -98.50405339999999}, {'lat': 29.4476302, 'lng': -98.5148396}, {'lat': 29.4481481, 'lng': -98.51495550000001}, {'lat': 30.5055786, 'lng': -99.7710946}, {'lat': 30.5087873, 'lng': -99.77427019999999}, {'lat': 31.21632, 'lng': -99.8456595}, {'lat': 31.3854594, 'lng': -100.3686965}, {'lat': 31.3876797, 'lng': -100.3731602}, {'lat': 31.4084142, 'lng': -100.3734915}, {'lat': 31.4775412, 'lng': -100.3862307}, {'lat': 31.4702126, 'lng': -100.4408002}, {'lat': 31.4682115, 'lng': -100.4458749}, {'lat': 31.4678186, 'lng': -100.4466502}, {'lat': 31.8538221, 'lng': -101.0335872}, {'lat': 32.2095324, 'lng': -101.5401878}, {'lat': 32.260929, 'lng': -101.5523919}, {'lat': 32.2650763, 'lng': -101.5513346}, {'lat': 32.2655119, 'lng': -101.5512538}, {'lat': 32.31

In [3]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Polyline

list_of_coords_south = [[d['lat'], d['lng']] for d in south_locs]
list_of_coords_north = [[d['lat'], d['lng']] for d in north_locs]

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(32.2540, -110.911789), #Set to Tuscon, AZ
    zoom=4.35
    )

route_south = Polyline(
    locations=list_of_coords_south,
    color="red",
    fill=False
)
route_north = Polyline(
    locations=list_of_coords_north,
    color="green",
    fill=False
)

m.add_layer(route_south)
m.add_layer(route_north)
m

Map(center=[32.254, -110.911789], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …